In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("..\\data\\kepler_KOI_full_dataset.csv")

In [6]:
df = df[df["kepler_name"].notna()]

In [9]:
df["kepler_name"].isnull().sum()

np.int64(0)

In [1]:
import requests
import os
import pandas as pd
from typing import List, Dict, Tuple, Any

BASE_URL = "https://archive-api.open-meteo.com/v1/archive"

def fetch_point_history(
    latitude: float,
    longitude: float,
    start_date: str,
    end_date: str,
    hourly_vars: List[str],
    daily_vars: List[str],
    timezone: str = "auto"
) -> Dict[str, Any]:
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": ",".join(hourly_vars) if hourly_vars else None,
        "daily": ",".join(daily_vars) if daily_vars else None,
        "timezone": timezone
    }
    # Remove None entries
    params = {k: v for (k, v) in params.items() if v is not None}
    resp = requests.get(BASE_URL, params=params, timeout=60)
    resp.raise_for_status()
    data = resp.json()
    if "error" in data:
        raise RuntimeError(f"API error: {data['error']}")
    return data

def make_daily_dataframe(
    location_id: int,
    country: str,
    city: str,
    lat: float,
    lon: float,
    data: Dict[str, Any],
    hourly_vars: List[str],
    daily_vars: List[str]
) -> pd.DataFrame:
    """
    Build a daily-aggregated DataFrame for one location, including the requested variables
    and the metadata columns like location_id, country, city, lat, lon.
    """
    daily = data.get("daily", {})
    df = pd.DataFrame(daily)
    # parse date
    df["date"] = pd.to_datetime(df["time"]).dt.date

    # From hourly, get weather_code if present
    if "weather_code" in data.get("hourly", {}):
        hh = pd.DataFrame(data["hourly"])
        hh["date"] = pd.to_datetime(hh["time"]).dt.date
        wc = hh.groupby("date")["weather_code"].first().reset_index()
        wc.rename(columns={"weather_code": "weather_code_daily"}, inplace=True)
        df = df.merge(wc, how="left", on="date")
        # rename to match final
        df.rename(columns={"weather_code_daily": "weather_code"}, inplace=True)

    # Add metadata columns
    df["location_id"] = location_id
    df["country"] = country
    df["city"] = city
    df["latitude"] = lat
    df["longitude"] = lon

    # You may want to reorder or drop extra columns
    # Ensure “time” (daily date string) is present, and “location_id” is first, etc.
    # For example:
    cols = ["location_id", "time", "date", "country", "city", "latitude", "longitude"]
    # Then all daily_vars + possibly weather_code
    if "weather_code" in df.columns:
        cols.append("weather_code")
    cols += daily_vars
    # Some columns might not exist (if API didn’t return them), so reindex
    df = df.reindex(columns=cols)
    return df

def save_df_to_csv(df: pd.DataFrame, filepath: str):
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    df.to_csv(filepath, index=False, encoding="utf-8")

def build_from_countries(
    country_points: Dict[str, List[Tuple[str, float, float]]],
    start_date: str,
    end_date: str,
    hourly_vars: List[str],
    daily_vars: List[str],
    output_base: str = "output_daily",
    master_csv_name: str = "all_locations_daily.csv",
    locations_csv_name: str = "locations.csv"
) -> None:
    """
    Fetch for all locations, save per-city daily CSVs, build master and lookup.
    """
    all_dfs = []  # list of DataFrames for all cities
    lookup = []   # list of dicts for location lookup table

    next_loc_id = 1
    for country, pts in country_points.items():
        for (city, lat, lon) in pts:
            loc_id = next_loc_id
            next_loc_id += 1

            print(f"Fetching for {country} / {city} …")
            try:
                data = fetch_point_history(lat, lon, start_date, end_date, hourly_vars, daily_vars)
            except Exception as e:
                print(f"  Error fetching {city}: {e}")
                continue

            df_daily = make_daily_dataframe(loc_id, country, city, lat, lon, data, hourly_vars, daily_vars)

            # Save per-city CSV under folder = country
            safe_city = city.replace(" ", "_")
            city_path = os.path.join(output_base, country, f"{safe_city}.csv")
            save_df_to_csv(df_daily, city_path)
            print(f"  Saved {city_path} (rows: {len(df_daily)})")

            all_dfs.append(df_daily)

            lookup.append({
                "location_id": loc_id,
                "country": country,
                "city": city,
                "latitude": lat,
                "longitude": lon
            })

    # Concatenate all city DataFrames
    if all_dfs:
        master = pd.concat(all_dfs, ignore_index=True)
        master_path = os.path.join(output_base, master_csv_name)
        save_df_to_csv(master, master_path)
        print(f"Master CSV saved to {master_path}, total rows: {len(master)}")

    # Save lookup table
    lookup_df = pd.DataFrame(lookup)
    lookup_path = os.path.join(output_base, locations_csv_name)
    save_df_to_csv(lookup_df, lookup_path)
    print(f"Locations lookup table saved to {lookup_path}, entries: {len(lookup_df)}")

if __name__ == "__main__":
    country_points = {
        "Greece": [("Athens", 37.9838, 23.7275), ("Thessaloniki", 40.6401, 22.9444), ("Heraklion", 35.3387, 25.1442)],
        "Spain": [("Madrid", 40.4168, -3.7038), ("Barcelona", 41.3851, 2.1734), ("Seville", 37.3891, -5.9845)],
        "Italy": [("Rome", 41.9028, 12.4964), ("Milan", 45.4642, 9.1900), ("Naples", 40.8518, 14.2681)]
    }
    start = "2001-01-01"
    end   = "2021-12-31"

    hourly_vars = ["weather_code"]
    daily_vars = [
        "temperature_2m_max",
        "temperature_2m_min",
        "apparent_temperature_max",
        "apparent_temperature_min",
        "sunrise",
        "sunset",
        "daylight_duration",
        "sunshine_duration",
        "precipitation_sum",
        "rain_sum",
        "snowfall_sum",
        "precipitation_hours",
        "wind_speed_10m_max",
        "wind_gusts_10m_max",
        "wind_direction_10m_dominant",
        "shortwave_radiation_sum",
        "et0_fao_evapotranspiration"
    ]

    build_from_countries(
        country_points,
        start,
        end,
        hourly_vars,
        daily_vars,
        output_base="weather_daily",
        master_csv_name="all_locations_daily.csv",
        locations_csv_name="locations.csv"
    )


Fetching for Greece / Athens …
  Saved weather_daily/Greece/Athens.csv (rows: 7670)
Fetching for Greece / Thessaloniki …
  Error fetching Thessaloniki: 429 Client Error: Too Many Requests for url: https://archive-api.open-meteo.com/v1/archive?latitude=40.6401&longitude=22.9444&start_date=2001-01-01&end_date=2021-12-31&hourly=weather_code&daily=temperature_2m_max%2Ctemperature_2m_min%2Capparent_temperature_max%2Capparent_temperature_min%2Csunrise%2Csunset%2Cdaylight_duration%2Csunshine_duration%2Cprecipitation_sum%2Crain_sum%2Csnowfall_sum%2Cprecipitation_hours%2Cwind_speed_10m_max%2Cwind_gusts_10m_max%2Cwind_direction_10m_dominant%2Cshortwave_radiation_sum%2Cet0_fao_evapotranspiration&timezone=auto
Fetching for Greece / Heraklion …
  Error fetching Heraklion: 429 Client Error: Too Many Requests for url: https://archive-api.open-meteo.com/v1/archive?latitude=35.3387&longitude=25.1442&start_date=2001-01-01&end_date=2021-12-31&hourly=weather_code&daily=temperature_2m_max%2Ctemperature_2m_